[Tutorial URL](https://python.langchain.com/docs/get_started/quickstart)

In [ ]:
!pipenv install langchain-openai
!pipenv install beautifulsoup4
!pipenv install langchain-community
!pipenv install faiss-cpu
!pipenv install langchain

In [ ]:
# Load the .env file
%load_ext dotenv
%dotenv

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [ ]:
llm.invoke("how can langsmith help with testing?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are world class technical documentation writer.'),
    ('user', '{input}')
])


In [ ]:
chain = prompt | llm
chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
vector

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>

                                          Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response['answer'])